In [ ]:
# Basic Q and A using OpenAI and Vector DB 

In [32]:
import os
import openai
from openai import OpenAI

In [ ]:
## Text Generation and Embeddings using OpenAI

In [8]:
def get_embedding(client, text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_response(client, preamble, prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f'{preamble}'},
            {"role": "user", "content": f'{prompt}'}]
    )
    return response

In [12]:
os.environ['OPENAI_API_KEY'] = "open_ai_key"
client = OpenAI()
preamble = "You are a famous song writer."
prompt = "Write a poem on football."
response = get_response(client, preamble, prompt)
# print(response)
text = response.choices[0].message.content
print(text)
# print(get_embedding(client, text))

In a world abuzz with cheers and clash,
Where passions ignite, where heroes dash,
There lies a game that stirs our soul,
A sport that unites in one grand goal.

Football, the field becomes a stage,
Where players dance, their talents engage,
Feet adorned with grace, they swiftly glide,
Across the pitch, with dreams held wide.

The roar of the crowd, a thunderous sound,
An anthem of unity, resounding and profound,
Fans from all walks of life, diverse and true,
United in their love for the game, through and through.

The ball, a sphere of hopes and dreams,
Precious like life’s most gleaming gleams,
Leaders emerge, bold captains of the team,
Guiding their comrades through victory's gleam.

With every pass, a symphony unfolds,
A choreography in which stories are told,
The ball weaves through the air with grace,
Creating a masterpiece upon the playing space.

Goalkeepers stand tall, defenders with might,
Rising to challenge, fearless in their fight,
Their resilience a fortress, built to with

In [ ]:
## Answering Questions from a Context

In [13]:
from datasets import load_dataset

In [14]:
dataset = load_dataset("godivyam/business-companies-news-dataset", split='train')

In [15]:
dataset[0]

{'Date': 'November 26, 2023',
 'Content': "The National Company Law Appellate Tribunal (NCLAT) adjourned the final hearing of tech giant Google's appeal against the order by Competition Commission of India (CCI) imposing a penalty of Rs 936 crore for abusing its dominant position in the playstore ecosystem.\nThe NCLAT released a notice on November 24 deferring the hearing of Google's appeal, the appellate tribunal was to commence the hearing on November 28. On September 25, theNCLAT announced that it will commence the final hearingof the case.\nOn January 11, thetribunal refused to grant immediate reliefto Google against CCI’s order and posted the case for the final hearing in April 2023. While Google filed an appeal at the Supreme Court against the NCLAT order, it ultimatelywithdrewthe case.\nThe CCI on October 25, 2022,penalisedthe tech giant for restricting app developers from using any third-party billing or payment processing services to purchase apps for in-app billing on Google 

In [16]:
preamble = dataset[0]['Content']
prompt = "What is Google's appeal to NCLAT?"
response = get_response(client, preamble, prompt)
text = response.choices[0].message.content
print(text)

Google's appeal to the NCLAT is against the order by the Competition Commission of India (CCI) imposing a penalty of Rs 936 crore for abusing its dominant position in the playstore ecosystem. Google is challenging the CCI's finding that it had restricted app developers from using any third-party billing or payment processing services to purchase apps for in-app billing on the Google Play Store. Google argues that it did not violate any competition laws and that its practices were justified.


In [ ]:
## Using a vector DB and Index

In [17]:
import chromadb

In [18]:
from chromadb.utils import embedding_functions

In [33]:
def return_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=text)
    return response["data"][0]["embedding"]

In [19]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ["OPENAI_API_KEY"],
                model_name="text-embedding-ada-002"
            )

In [20]:
client = chromadb.Client()
collection = client.get_or_create_collection("companies-news",embedding_function=openai_ef)

In [23]:
docs = []
ids = []
for i,doc in enumerate(dataset):
    docs.append(doc['Content'])
    ids.append(str(i))

In [24]:
collection.add(
    documents=docs,
    ids=ids
)

In [25]:
query = "google appeal NCLAT"
prompt = "What is Google's appeal to NCLAT?"

In [35]:
openai_client = OpenAI()

In [38]:
vector=get_embedding(openai_client, query)
results=collection.query(
    query_embeddings=vector,
    n_results=2,
    include=["documents"]
)

In [39]:
results

{'ids': [['0', '16']],
 'distances': None,
 'metadatas': None,
 'embeddings': None,
 'documents': [["The National Company Law Appellate Tribunal (NCLAT) adjourned the final hearing of tech giant Google's appeal against the order by Competition Commission of India (CCI) imposing a penalty of Rs 936 crore for abusing its dominant position in the playstore ecosystem.\nThe NCLAT released a notice on November 24 deferring the hearing of Google's appeal, the appellate tribunal was to commence the hearing on November 28. On September 25, theNCLAT announced that it will commence the final hearingof the case.\nOn January 11, thetribunal refused to grant immediate reliefto Google against CCI’s order and posted the case for the final hearing in April 2023. While Google filed an appeal at the Supreme Court against the NCLAT order, it ultimatelywithdrewthe case.\nThe CCI on October 25, 2022,penalisedthe tech giant for restricting app developers from using any third-party billing or payment processi

In [40]:
context = "\n".join(str(item) for item in results['documents'][0])

In [41]:
preamble = f'Please answer using the context below. {context}'

In [42]:
response = get_response(openai_client, preamble, prompt)
text = response.choices[0].message.content
print(text)

Google's appeal to the National Company Law Appellate Tribunal (NCLAT) is against the order by the Competition Commission of India (CCI) imposing a penalty of Rs 936 crore on Google for abusing its dominant position in the playstore ecosystem.
